### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [1]:
!pwd

/Users/diat.lov/GitHub/RecSys-hse-fall-2021/src/hw1


In [2]:
%load_ext autoreload
%autoreload 2

import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp
import sys
import scipy

from resources import DATA_PATH
from mse_logs import LOGGING_PATH

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [3]:
ratings = pd.read_csv(f'{DATA_PATH.parent}/ratings.dat', delimiter='::', header=None, 
                      names=['user_id', 'movie_id', 'rating', 'timestamp'], 
                      usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [4]:
movie_info = pd.read_csv(f'{DATA_PATH.parent}/movies.dat', delimiter='::', header=None, 
                         names=['movie_id', 'name', 'category'], engine='python')

In [309]:
2000 * 15000

30000000

In [311]:
np.einsum('i, ij -> ij', np.ones(5), np.ones((16, 5)).T).shape

(5, 16)

In [314]:
np.dot(np.ones(5), np.ones((16, 5)).T).shape

(16,)

Explicit данные

In [91]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [302]:
movie_info.movie_id.unique().shape[0]

3883

In [293]:
users = ratings["user_id"]
movies = ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

In [294]:
user_item_csr.toarray().shape

(6041, 3953)

Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [59]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [60]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [68]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

In [69]:
user_item_csr.toarray().shape

(6041, 3953)

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

In [ ]:
model.fit(user_item_t_csr)

В качестве loss здесь всеми любимый RMSE

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [16]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [11]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '1838    Mulan (1998)',
 '1526    Hercules (1997)',
 '2618    Tarzan (1999)',
 '2692    Iron Giant, The (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [12]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [13]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [14]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [15]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '2502    Matrix, The (1999)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1892    Rain Man (1988)',
 '3402    Close Encounters of the Third Kind (1977)',
 '1179    Princess Bride, The (1987)',
 '847    Godfather, The (1972)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

## Data Observation

In [4]:
ratings = pd.read_csv(f'{DATA_PATH.parent}/ratings.dat', delimiter='::', header=None, 
                      names=['user_id', 'movie_id', 'rating', 'timestamp'], 
                      usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [5]:
grouped_df = ratings.groupby('user_id').count()

In [7]:
print(f'mean and std, N ratings: {np.floor(grouped_df.movie_id.mean())}, {np.floor(grouped_df.movie_id.std())}')

mean and std, N ratings: 165.0, 192.0


### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [24]:
# from svd.svd import SVD
from scipy import spatial
# from svd.svd_no_mapping import SVD
from svd.simple_svd import SVD

In [25]:
users = ratings.user_id
movies = ratings.movie_id
user_item = sp.coo_matrix((ratings.rating, (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

In [ ]:
svd_model = SVD(user_item_csr)
user_matrix, item_matrix = svd_model.fit(logging_path=LOGGING_PATH.parent)

Start fitting the model...


##### Посмотрим на схожие фильмы

In [348]:
# Toy Story

In [8]:
movie_info.loc[[0]]

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy


In [9]:
movie_info.set_index('movie_id').loc[svd_model.get_k_similar_movies(np.array([1, 1]))[0]]

,name,category
movie_id,,
1,Toy Story (1995),Animation|Children's|Comedy
1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
2028,Saving Private Ryan (1998),Action|Drama|War
1500,Grosse Pointe Blank (1997),Comedy|Crime
1307,When Harry Met Sally... (1989),Comedy|Romance


In [10]:
movie_info.set_index('movie_id').loc[[1291]]

,name,category
movie_id,,
1291,Indiana Jones and the Last Crusade (1989),Action|Adventure


In [11]:
movie_info.set_index('movie_id').loc[svd_model.get_k_similar_movies(np.array([1291, 1291]))[0]]

,name,category
movie_id,,
1291,Indiana Jones and the Last Crusade (1989),Action|Adventure
1265,Groundhog Day (1993),Comedy|Romance
1198,Raiders of the Lost Ark (1981),Action|Adventure
1036,Die Hard (1988),Action|Thriller
2174,Beetlejuice (1988),Comedy|Fantasy


In [216]:
predictions = svd_model.predict_top_k(ratings.user_id.unique()[:100], top_k=10)

In [217]:
svd_model.recall_at_k(predictions, ratings.user_id.unique()[:100])

0.06430000000000001

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных